In [135]:
import pandas as pd 
pd.options.mode.chained_assignment = None
from pathlib import Path 
import numpy as np
from typing import Union 
from datetime import datetime, timedelta
from queue import PriorityQueue
import re
from timeit import default_timer as timer

DATA_DIR = Path('../data')

In [3]:
connection_graph = pd.read_csv(DATA_DIR / 'connection_graph_latest.csv', 
                               usecols=['line', 'departure_time', 'arrival_time', 'start_stop', 'end_stop'], 
                               dtype=str)

In [4]:
connection_graph.dtypes

line              object
departure_time    object
arrival_time      object
start_stop        object
end_stop          object
dtype: object

In [5]:
connection_graph.columns

Index(['line', 'departure_time', 'arrival_time', 'start_stop', 'end_stop'], dtype='object')

In [6]:
connection_graph.head()

,line,departure_time,arrival_time,start_stop,end_stop
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska)
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego


In [7]:
len(connection_graph)

996520

In [8]:
connection_graph = connection_graph.drop_duplicates()

In [9]:
len(connection_graph)

455110

Obserwacja: niektóre linie są wyrażone jako int, a niektóre jako str - najlepiej to sprowadzić to wspólnego formatu str

In [10]:
connection_graph['line'].unique()

array(['A', 'D', 'K', 'N', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       '21', '22', '23', '100', '101', '102', '103', '104', '105', '106',
       '107', '108', '110', '111', '112', '113', '114', '115', '116 ',
       '117', '118', '119', '120', '121', '122', '123', '124', '125',
       '126', '127', '128 ', '129', '130', '131', '132', '133', '134',
       '136', '137', '138', '142', '143', '144', '145', '146', '147',
       '148', '149', '150', '151 ', '152', '206', '240', '241', '242',
       '243', '244', '245', '246', '247', '248', '249', '250', '251',
       '253', '255', '257', '259', '310', '315', '319', '345', '602',
       '607', '612', '714', '903', '904', '907', '908', '909', '911',
       '913', '914', '917', '920', '921', '923', '924', '927', '930',
       '931', '933', '934', '936', '937', '938', '941', '947', '948',
       '958', '967'], dtype=object)

Założenie: czas przejazdu między przystankami dla tej samej linii może być różny między różnymi godzinami 

In [11]:
krzyki_sowia_conn = connection_graph[(connection_graph['start_stop'] == 'KRZYKI') & (connection_graph['end_stop'] == 'Sowia')]

In [12]:
krzyki_sowia_conn['line'].unique()

array(['A', '126', '248', '251'], dtype=object)

In [13]:
duplicated_stops = connection_graph[connection_graph['start_stop'] == connection_graph['end_stop']]
duplicated_stops.head()

,line,departure_time,arrival_time,start_stop,end_stop
14127,D,08:43:00,08:44:00,Pomorska,Pomorska
15247,D,09:25:00,09:26:00,Pomorska,Pomorska
15934,D,20:49:00,20:50:00,Pomorska,Pomorska
16934,D,20:12:00,20:13:00,Pomorska,Pomorska
21374,D,05:46:00,05:47:00,Pomorska,Pomorska


POKAZAC ZE DLA TAKIEGO PRZYPADKU BEDZIE DZIALALO

W przypadku implementacji Dijkstry to, że istnieją połączenia między tym samym przystankiem nie będzie miało znaczenia, tj. nie będziemy takich połączeń rozważać

In [14]:
connection_graph.loc[duplicated_stops.iloc[0].name - 5:duplicated_stops.iloc[0].name + 5]

,line,departure_time,arrival_time,start_stop,end_stop
14122,D,08:34:00,08:35:00,Obornicka (Wołowska),Bezpieczna
14123,D,08:35:00,08:36:00,Bezpieczna,Bałtycka
14124,D,08:36:00,08:39:00,Bałtycka,Kleczkowska
14125,D,08:39:00,08:40:00,Kleczkowska,pl. Strzelecki
14126,D,08:40:00,08:43:00,pl. Strzelecki,Pomorska
14127,D,08:43:00,08:44:00,Pomorska,Pomorska
14128,D,08:44:00,08:45:00,Pomorska,Mosty Pomorskie
14129,D,08:45:00,08:47:00,Mosty Pomorskie,Rynek
14130,D,08:47:00,08:49:00,Rynek,Narodowe Forum Muzyki
14131,D,08:49:00,08:51:00,Narodowe Forum Muzyki,Renoma


Problem dotyczący dat:
- format nie jest jednolity, tj. istnieją godziny 25
- porównywanie godzin nie daje żądanych wyników, tj. chcielibyśmy, by godzina 00:00 była po 23:59

In [15]:
(datetime.strptime('00:00:00', '%H:%M:%S') > datetime.strptime('23:59:00', '%H:%M:%S'))

False

In [16]:
def separate_time(time):
    return [int(tp) for tp in time.split(':')]

In [17]:
separate_time('20:00:00')

[20, 0, 0]

In [18]:
connection_graph[connection_graph['departure_time'].apply(lambda x: separate_time(x)[0]) >= 24].head()

,line,departure_time,arrival_time,start_stop,end_stop
30969,K,24:01:00,24:03:00,Broniewskiego,Bałtycka
30970,K,24:03:00,24:04:00,Bałtycka,Bezpieczna
30971,K,24:04:00,24:05:00,Bezpieczna,Paprotna
30972,K,24:05:00,24:06:00,Paprotna,Zajezdnia Obornicka
31418,K,24:00:00,24:03:00,DWORZEC GŁÓWNY,GALERIA DOMINIKAŃSKA


In [19]:
connection_graph[connection_graph['departure_time'].apply(lambda x: separate_time(x)[0]) >= 24].max()

line                     K
departure_time    30:03:00
arrival_time      30:05:00
start_stop         Żmudzka
end_stop           Żmudzka
dtype: object

In [20]:
connection_graph[connection_graph['departure_time'].apply(lambda x: separate_time(x)[0]) <= 4].min()

line                     1
departure_time    03:34:00
arrival_time      03:36:00
start_stop          8 Maja
end_stop            8 Maja
dtype: object

1. Czas zostanie znormalizowany tak, by wszystkie godziny były w przedziale [0:00; 23:59]

In [21]:
def to_seconds(time: str) -> int:
    h, m, s = separate_time(time)
    return int(h * 3600 + m * 60 + s)

def time_to_normalized_sec(time: str) -> int:
    return to_seconds(time) % (3600 * 24)

In [22]:
connection_graph['departure_sec'] = connection_graph['departure_time'].apply(time_to_normalized_sec)
connection_graph['arrival_sec'] = connection_graph['arrival_time'].apply(time_to_normalized_sec)

In [23]:
connection_graph['arrival_sec']

0         75180
1         75240
2         75300
3         75420
4         75540
          ...  
996209    67140
996210    67260
996211    67320
996212    67380
996213    67500
Name: arrival_sec, Length: 455110, dtype: int64

In [24]:
print(connection_graph['departure_sec'].max() <= (24 * 3600))
print(connection_graph['arrival_sec'].max() <= (24 * 3600))

True
True


2. Stworzymy funkcję, która będzie respektować porządek godzinowy, nie liczbowy

In [141]:
def diff(ts: Union[pd.Series, int], td: int) -> int:
    '''Function that returns difference between ts and td times expressed as seconds 
        - note that this will never be a negative value'''
    d = ts - td 
    if isinstance(ts, pd.Series):
        d[d < 0] += 24 * 3600
    elif d < 0:
        d += 24 * 3600
    return d 

In [26]:
t_01_00 = 1 * 3600
t_22_00 = 22 * 3600
t_23_00 = 23 * 3600
t_00_01 = 0 + 60
t_21_00 = 21 * 3600
t_12_00 = 12 * 3600

print('01:00 - 22:00 = ', timedelta(seconds=diff(t_01_00, t_22_00)))
print('01:00 - 12:00 = ', timedelta(seconds=diff(t_01_00, t_12_00)))
print('23:00 - 22:00 = ', timedelta(seconds=diff(t_23_00, t_22_00)))
print('00:01 - 22:00 = ', timedelta(seconds=diff(t_00_01, t_22_00)))
print('23:00 - 12:00 = ', timedelta(seconds=diff(t_23_00, t_12_00)))
print('12:00 - 22:00 = ', timedelta(seconds=diff(t_12_00, t_22_00)))


01:00 - 22:00 =  3:00:00
01:00 - 12:00 =  13:00:00
23:00 - 22:00 =  1:00:00
00:01 - 22:00 =  2:01:00
23:00 - 12:00 =  11:00:00
12:00 - 22:00 =  14:00:00


In [27]:
t_20_00 = to_seconds('20:00:00')
t_20_01 = to_seconds('20:01:00')
t_20_02 = to_seconds('20:02:00')
t_19_59 = to_seconds('19:59:00')
t_20_03 = to_seconds('20:03:00')

print('19:59:00 - 20:00:00 = ', timedelta(seconds=diff(t_19_59, t_20_00)))
print('20:01:00 - 20:00:00 = ', timedelta(seconds=diff(t_20_01, t_20_00)))
print('20:02:00 - 20:00:00 = ', timedelta(seconds=diff(t_20_02, t_20_00)))
print('20:03:00 - 20:00:00 = ', timedelta(seconds=diff(t_20_03, t_20_00)))

19:59:00 - 20:00:00 =  23:59:00
20:01:00 - 20:00:00 =  0:01:00
20:02:00 - 20:00:00 =  0:02:00
20:03:00 - 20:00:00 =  0:03:00


In [28]:
krzyki_sowia_conn = connection_graph[(connection_graph['start_stop'] == 'KRZYKI') & (connection_graph['end_stop'] == 'Sowia')]
krzyki_sowia_conn.head()

,line,departure_time,arrival_time,start_stop,end_stop,departure_sec,arrival_sec
44,A,22:19:00,22:20:00,KRZYKI,Sowia,80340,80400
169,A,22:00:00,22:01:00,KRZYKI,Sowia,79200,79260
241,A,06:38:00,06:39:00,KRZYKI,Sowia,23880,23940
305,A,09:08:00,09:09:00,KRZYKI,Sowia,32880,32940
369,A,11:25:00,11:26:00,KRZYKI,Sowia,41100,41160


Przykład: dostajemy prawidłowe, logiczne sortowanie wg. godzin odjazdu

In [29]:
sorted_by_dep = krzyki_sowia_conn.sort_values(by='departure_sec')

In [30]:
sorted_by_dep

,line,departure_time,arrival_time,start_stop,end_stop,departure_sec,arrival_sec
911433,248,24:13:00,24:14:00,KRZYKI,Sowia,780,840
916404,251,24:23:00,24:24:00,KRZYKI,Sowia,1380,1440
911275,248,25:13:00,25:14:00,KRZYKI,Sowia,4380,4440
916115,251,25:23:00,25:24:00,KRZYKI,Sowia,4980,5040
911504,248,26:13:00,26:14:00,KRZYKI,Sowia,7980,8040
...,...,...,...,...,...,...,...
641864,126,22:18:00,22:19:00,KRZYKI,Sowia,80280,80340
44,A,22:19:00,22:20:00,KRZYKI,Sowia,80340,80400
2047,A,22:34:00,22:35:00,KRZYKI,Sowia,81240,81300
644431,126,22:46:00,22:47:00,KRZYKI,Sowia,81960,82020


Warto zauważyć, co może jest oczywsite, że do danego przystanku możemy się dostać wieloma liniami. Będzie to miało znaczenie takie, że mając do wyboru wiele linii do następnego przystanku możemy wybrać tą, którą najszybciej się dostaniemy (tj. arrival_time będzie najmniejszy), ponieważ zawsze możemy się przesiąść - więc nawet jeśli wybierzemy linię, która nie prowadzi nas do celu bezpośrednio, to możemy się na dowolnym przystanku przesiąść w prawidłową linię. 

In [31]:
connection_graph[connection_graph['start_stop'] == 'KRZYKI'][['line', 'end_stop']].drop_duplicates()

,line,end_stop
44,A,Sowia
14074,D,Zimowa
14153,D,Orla
64771,2,Orla
127704,6,Orla
139301,7,Radio i Telewizja
139311,7,Zajezdnia BOREK
139375,7,Orla
246676,14,Orla
283710,17,Radio i Telewizja


In [32]:
end_stops = connection_graph[(connection_graph['start_stop'] == 'KRZYKI')]['end_stop'].unique()

In [33]:
from_sliczna_143 = connection_graph[(connection_graph['start_stop'] == 'Śliczna') & (connection_graph['end_stop'] == 'Uniwersytet Ekonomiczny') & (connection_graph['line'] == '143')]

In [34]:
from_sliczna_143.sort_values(by='departure_sec')

,line,departure_time,arrival_time,start_stop,end_stop,departure_sec,arrival_sec
802911,143,05:12:00,05:15:00,Śliczna,Uniwersytet Ekonomiczny,18720,18900
795984,143,05:13:00,05:16:00,Śliczna,Uniwersytet Ekonomiczny,18780,18960
805822,143,05:34:00,05:37:00,Śliczna,Uniwersytet Ekonomiczny,20040,20220
797068,143,05:43:00,05:46:00,Śliczna,Uniwersytet Ekonomiczny,20580,20760
802361,143,05:52:00,05:55:00,Śliczna,Uniwersytet Ekonomiczny,21120,21300
...,...,...,...,...,...,...,...
795308,143,21:05:00,21:08:00,Śliczna,Uniwersytet Ekonomiczny,75900,76080
795907,143,21:35:00,21:38:00,Śliczna,Uniwersytet Ekonomiczny,77700,77880
796912,143,22:05:00,22:08:00,Śliczna,Uniwersytet Ekonomiczny,79500,79680
798266,143,22:35:00,22:38:00,Śliczna,Uniwersytet Ekonomiczny,81300,81480


In [35]:
type(end_stops)

numpy.ndarray

In [128]:
class Graph:

    def __init__(self, conn_graph, change_time=0):
        self.conn_graph = conn_graph
        self.change_time = change_time
        self.transform()

    def transform(self):
        '''Applies time transformation functions to departure and arrival times - the results are available in new columns'''
        self.conn_graph = self.conn_graph.drop_duplicates()
        self.conn_graph.loc[:, 'departure_sec'] = self.conn_graph['departure_time'].apply(time_to_normalized_sec)
        self.conn_graph.loc[:, 'arrival_sec'] = self.conn_graph['arrival_time'].apply(time_to_normalized_sec)

    def get_start_idx(self):
        return -1

    def add_start_conn(self, dep_time: int, start_stop: str):
        self.conn_graph.loc[self.get_start_idx()] = pd.Series({'end_stop': start_stop, 'departure_sec': dep_time, 'arrival_sec': dep_time})

    def remove_start_conn(self):
        self.conn_graph.drop(self.get_start_idx(), inplace=True)

    def get_nodes(self):
        '''should we also add end_stop?'''
        return self.conn_graph['start_stop'].unique()
    
    def get_lines(self, start_stop, end_stop):
        '''Returns lines connecting two stops'''
        return self.conn_graph[(self.conn_graph['start_stop'] == start_stop) 
                               & (self.conn_graph['end_stop'] == end_stop)]['line'].unique()
    
    def get_stops(self, line):
        '''? this may not make sense because sometimes the routes are different'''
        return self.conn_graph[self.conn_graph['line'] == line]['start_stop'].unique()

    def get_neighbour_stops(self, start_stop: str) -> np.array:
        '''Returns neighbouring end stops'''
        # return self.conn_graph[self.conn_graph['start_stop'] == start_stop][['line', 'end_stop']].drop_duplicates()
        return self.conn_graph[self.conn_graph['start_stop'] == start_stop]['end_stop'].unique()

    def get_conn_valid_time_arrivals(self, dep_time: int, start_stop: str, end_stop: str) -> pd.Series:
        conn = self.conn_graph[(self.conn_graph['start_stop'] == start_stop) 
                               & (self.conn_graph['end_stop'] == end_stop)]
 
        time_arrv_diff = diff(conn['arrival_sec'], dep_time)

        # Here we should add some optional time for changing lines, but we would also need to
        # pass the current line
        time_dep_diff = diff(conn['departure_sec'], dep_time)
        # Sort by the departure time, and for the same departure time favour the connection 
        # with the earliest arrival
        differences = (time_arrv_diff - time_dep_diff) >= 0
        valid_time_arrv_diff = time_arrv_diff[differences]
        return conn, valid_time_arrv_diff

    def get_sorted_conns(self, dep_time: int, start_stop: str, end_stop: str):
        '''get all connections sorted by time arrival'''
        conn, valid_time_arrv_diff = self.get_conn_valid_time_arrivals(dep_time, start_stop, end_stop)

        return conn.loc[valid_time_arrv_diff.sort_values().index]


    def get_earliest_conn(self, dep_time: int, start_stop: str, end_stop: str) -> pd.Series:
        '''Returns the earliest connection between two stops'''
        conn, valid_time_arrv_diff = self.get_conn_valid_time_arrivals(dep_time, start_stop, end_stop)

        return conn.loc[valid_time_arrv_diff.idxmin()]
    
    def get_earliest_from(self, dep_time: int, start_stop: str):
        '''Returns all earliest connections to all neighbouring stops'''
        return [self.get_earliest_conn(dep_time, start_stop, end_stop) for 
                end_stop in self.get_neighbour_stops(start_stop)]

    def sub_cost(self, dep_time: int, next_idx: int, curr_idx: int = None) -> int:
        cost = diff(self.conn_graph.loc[next_idx, 'arrival_sec'], dep_time)
        # cost += self.change_time if current['line'] is not None and current['line'] != next['line'] else 0
        return cost
    
    def conn_at_index(self, index: int) -> pd.Series:
        return self.conn_graph.loc[index] 

In [101]:
g = Graph(connection_graph)

In [77]:
dep_time = 8 * 3600

In [78]:
g.add_start_conn(dep_time, 'PL. GRUNWALDZKI')

In [79]:
g.conn_at_index(-1)

line                          NaN
departure_time                NaN
arrival_time                  NaN
start_stop                    NaN
end_stop          PL. GRUNWALDZKI
departure_sec               28800
arrival_sec                 28800
Name: -1, dtype: object

In [80]:
g.conn_at_index(71821)

line                                              2
departure_time                             08:02:00
arrival_time                               08:04:00
start_stop                          PL. GRUNWALDZKI
end_stop          Kliniki - Politechnika Wrocławska
departure_sec                                 28920
arrival_sec                                   29040
Name: 71821, dtype: object

In [84]:
# cost for the first node
g.sub_cost(dep_time, 71821, -1)

240

In [82]:
g.get_neighbour_stops('PL. GRUNWALDZKI')

array(['most Grunwaldzki', 'Kochanowskiego',
       'Kliniki - Politechnika Wrocławska', 'Piastowska', 'Reja',
       'Bujwida', 'PL. GRUNWALDZKI'], dtype=object)

In [66]:
g.get_neighbour_stops('Kliniki - Politechnika Wrocławska')

array(['Hala Stulecia', 'PL. GRUNWALDZKI', 'Mickiewicza'], dtype=object)

In [67]:
conn = connection_graph[(connection_graph['start_stop'] == 'PL. GRUNWALDZKI') & (connection_graph['end_stop'] == 'Kliniki - Politechnika Wrocławska')]

time_diff = diff(conn['departure_sec'], dep_time)
conn.loc[time_diff.sort_values().index].head(10)

,line,departure_time,arrival_time,start_stop,end_stop,departure_sec,arrival_sec
71821,2,08:02:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28920,29040
320086,19,08:03:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28980,29040
543863,115,08:03:00,08:05:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28980,29100
60334,1,08:04:00,08:06:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,29040,29160
99119,4,08:06:00,08:08:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,29160,29280
851072,146,08:07:00,08:09:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,29220,29340
191231,10,08:07:00,08:09:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,29220,29340
834618,145,08:10:00,08:12:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,29400,29520
184511,10,08:10:00,08:12:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,29400,29520
548622,115,08:10:00,08:12:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,29400,29520


In [68]:
time_arrv_diff = diff(conn['arrival_sec'], dep_time)
time_dep_diff = diff(conn['departure_sec'], dep_time)

In [69]:
# We want both arrival and departure time to be one the 'same' side - that is, they are either after the 
# given dept_time or before
differences = (time_arrv_diff - time_dep_diff) >= 0

In [70]:
valid_time_arrv_diff = time_arrv_diff[differences]
conn.loc[valid_time_arrv_diff.idxmin()]

line                                              2
departure_time                             08:02:00
arrival_time                               08:04:00
start_stop                          PL. GRUNWALDZKI
end_stop          Kliniki - Politechnika Wrocławska
departure_sec                                 28920
arrival_sec                                   29040
Name: 71821, dtype: object

In [71]:
pd.concat([conn['departure_time'], conn['arrival_time'], time_dep_diff, time_arrv_diff, differences], axis=1).loc[time_arrv_diff.sort_values().index]

,departure_time,arrival_time,departure_sec,arrival_sec,0
54062,08:01:00,08:02:00,86340,0,False
551172,08:00:00,08:02:00,86280,0,False
325694,08:01:00,08:03:00,86340,60,False
320086,08:03:00,08:04:00,60,120,True
71821,08:02:00,08:04:00,0,120,True
...,...,...,...,...,...
193979,07:55:00,07:57:00,85980,86100,True
845919,07:55:00,07:57:00,85980,86100,True
66956,07:56:00,07:58:00,86040,86160,True
840500,07:59:00,08:01:00,86220,86340,True


In [72]:
conn[differences].loc[time_arrv_diff[differences].sort_values().index]

,line,departure_time,arrival_time,start_stop,end_stop,departure_sec,arrival_sec
320086,19,08:03:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28980,29040
71821,2,08:02:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28920,29040
543863,115,08:03:00,08:05:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28980,29100
60334,1,08:04:00,08:06:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,29040,29160
99119,4,08:06:00,08:08:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,29160,29280
...,...,...,...,...,...,...,...
193979,10,07:55:00,07:57:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28500,28620
845919,146,07:55:00,07:57:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28500,28620
66956,2,07:56:00,07:58:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28560,28680
840500,145,07:59:00,08:01:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28740,28860


In [73]:
g.get_earliest_conn(dep_time, 'PL. GRUNWALDZKI', 'Kliniki - Politechnika Wrocławska')

line                                              2
departure_time                             08:02:00
arrival_time                               08:04:00
start_stop                          PL. GRUNWALDZKI
end_stop          Kliniki - Politechnika Wrocławska
departure_sec                                 28920
arrival_sec                                   29040
Name: 71821, dtype: object

In [74]:
g.get_sorted_conns(dep_time, 'PL. GRUNWALDZKI', 'Kliniki - Politechnika Wrocławska')

,line,departure_time,arrival_time,start_stop,end_stop,departure_sec,arrival_sec
320086,19,08:03:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28980,29040
71821,2,08:02:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28920,29040
543863,115,08:03:00,08:05:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28980,29100
60334,1,08:04:00,08:06:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,29040,29160
99119,4,08:06:00,08:08:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,29160,29280
...,...,...,...,...,...,...,...
193979,10,07:55:00,07:57:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28500,28620
845919,146,07:55:00,07:57:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28500,28620
66956,2,07:56:00,07:58:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28560,28680
840500,145,07:59:00,08:01:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28740,28860


In [75]:
g.conn_graph.loc[71821:71822]

,line,departure_time,arrival_time,start_stop,end_stop,departure_sec,arrival_sec
71821,2,08:02:00,08:04:00,PL. GRUNWALDZKI,Kliniki - Politechnika Wrocławska,28920,29040
71822,2,08:04:00,08:06:00,Kliniki - Politechnika Wrocławska,Hala Stulecia,29040,29160


Od 8:00 dostaniemy się o 8:04, więc czas przejazdu - czyli koszt - to 240 sekund

In [86]:
g.sub_cost(dep_time, 71821)

240

In [87]:
frontier = PriorityQueue()
frontier.put((10, 1))
frontier.put((5, 3))
print(frontier.get())
frontier.put((5, 3))
frontier.put((2, 8))
print(frontier.get())

(5, 3)
(2, 8)


In [88]:
frontier.get()

(5, 3)

Krotka jest w formacie (priority, data)

In [89]:
def print_info(conn, file=None):
    print(f"{conn['line']} goes from {conn['start_stop']} to {conn['end_stop']} and leaves at {conn['departure_time']}, arrives at {conn['arrival_time']}", 
          file=file)

In [90]:
def sec_to_time(seconds: int) -> str:
    hour = (seconds // 3600)
    minutes = (seconds % 3600) // 60 
    secs = (seconds % 3600) % 60 
    return f'{hour:02d}:{minutes:02d}:{secs:02d}'

In [129]:
def find_path(graph: Graph, start_stop: str, goal_stop: str, leave_hour: str):
    
    frontier = PriorityQueue()
    dep_time = time_to_normalized_sec(leave_hour)
    graph.add_start_conn(dep_time, start_stop)

    cost_so_far = {}
    # if commuting A -> B, then this will be came_from[B] = A so we can recreate the path
    came_from = {}
    cost_so_far[start_stop] = 0
    came_from[graph.get_start_idx()] = None

    frontier.put((cost_so_far[start_stop], graph.get_start_idx()))

    with open(DATA_DIR / ('dijkstra_runs/' + re.sub(r"\W+", "", start_stop) + '-' + re.sub(r"\W+", "", goal_stop)), mode='w', encoding='utf-8') as f:

        i = 0
        while not frontier.empty():
            # get the stop with the lowest cost 
            cost, index = frontier.get()

            conn = graph.conn_at_index(index)
            current = conn['end_stop']

            if current == goal_stop:
                goal_index = index
                break 

            print(f'[{i}]', file=f)
            for next_conn in graph.get_earliest_from(dep_time=dep_time + cost, start_stop=current):
                print_info(next_conn, file=f)
                next = next_conn['end_stop']
                # cost of commuting start --> current and current --> next 
                new_cost = cost + graph.sub_cost(dep_time + cost, next_conn.name, conn.name)
                if next not in cost_so_far or new_cost < cost_so_far[next]:
                    print(f'We arrive from {start_stop} to {next} at {sec_to_time(dep_time + new_cost)}', file=f)
                    cost_so_far[next] = new_cost
                    frontier.put((new_cost, next_conn.name))
                    came_from[next_conn.name] = index
            i+=1

    return goal_index, came_from, cost_so_far

In [130]:
def path_to_list(node: str, connections: dict):
    path = [node]
    while node:
        node = connections[node]
        path.append(node)
    path.reverse()
    return path[1:]

def idxs_to_nodes(graph: Graph, goal_idx: int, conn_idxs: dict):
    idx_path = path_to_list(goal_idx, conn_idxs)
    return [graph.conn_at_index(idx) for idx in idx_path[1:]]

def print_path(connections: dict):
    for conn in connections:
        print(f'{conn["start_stop"]} [{conn["departure_time"]}] --- {conn["line"]} ---> {conn["end_stop"]} [{conn["arrival_time"]}]')

In [131]:
def run_solution(start_stop: str, goal_stop: str, leave_hour: str, change_time=0):
    start = timer()
    connection_graph = pd.read_csv(DATA_DIR / 'connection_graph.csv', 
                               usecols=['line', 'departure_time', 'arrival_time', 'start_stop', 'end_stop'], 
                               dtype=str)
    graph = Graph(connection_graph, change_time)
    goal_index, came_from, costs = find_path(graph, start_stop, goal_stop, leave_hour)
    end = timer()
    elapsed_time = (end - start)
    solution_cost = costs[goal_stop]
    return graph, goal_index, came_from, solution_cost, elapsed_time, 

In [143]:
def dijkstra(start_stop: str, goal_stop: str, leave_hour: str, change_time=0):
    print(f'Commute from {start_stop} to {goal_stop} at {leave_hour}')
    graph, goal_index, came_from, solution_cost, elapsed_time = run_solution(start_stop, goal_stop, leave_hour, change_time)
    connections = idxs_to_nodes(graph, goal_index, came_from)
    print_path(connections)
    print(f'Total trip time is {sec_to_time(solution_cost)}')
    commuting_time = sec_to_time(diff(connections[-1]['arrival_sec'], connections[0]['departure_sec']))
    print(f'Commuting takes {commuting_time}')
    print(f'Algorithm took {elapsed_time:.2f}s to execute')

Przypadki testowe:


In [133]:
test_cases = pd.read_json(DATA_DIR / 'test_cases.json')
test_cases['dept_time'] = test_cases['dept_time'].apply(lambda x: x.strftime('%H:%M:%S'))

test_cases = test_cases.values

In [134]:
dijkstra(*test_cases[0])

C:\Users\julia\AppData\Local\Temp\ipykernel_36600\2860909127.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.conn_graph.loc[:, 'departure_sec'] = self.conn_graph['departure_time'].apply(time_to_normalized_sec)
C:\Users\julia\AppData\Local\Temp\ipykernel_36600\2860909127.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.conn_graph.loc[:, 'arrival_sec'] = self.conn_graph['arrival_time'].apply(time_to_normalized_sec)
C:\Users\julia\AppData\Local\Temp\ipykernel_36600\2860909127.py:18: SettingW

Commute from KLECINA to OSIEDLE SOBIESKIEGO
KLECINA [20:02:00] --- 107 ---> Skarbowców [20:03:00]
Skarbowców [20:03:00] --- 107 ---> Os. Przyjaźni [20:04:00]
Os. Przyjaźni [20:04:00] --- 107 ---> Zimowa [20:05:00]
Zimowa [20:05:00] --- 107 ---> KRZYKI [20:06:00]
KRZYKI [20:08:00] --- 7 ---> Orla [20:10:00]
Orla [20:10:00] --- 7 ---> Jastrzębia [20:11:00]
Jastrzębia [20:11:00] --- 7 ---> Hallera [20:12:00]
Hallera [20:12:00] --- 7 ---> Sztabowa [20:13:00]
Sztabowa [20:13:00] --- 7 ---> Rondo [20:14:00]
Rondo [20:14:00] --- 7 ---> Wielka [20:15:00]
Wielka [20:15:00] --- 7 ---> Zaolziańska [20:16:00]
Zaolziańska [20:16:00] --- 7 ---> Arkady (Capitol) [20:19:00]
Arkady (Capitol) [20:24:00] --- D ---> GALERIA DOMINIKAŃSKA [20:29:00]
GALERIA DOMINIKAŃSKA [20:29:00] --- D ---> Urząd Wojewódzki (Impart) [20:31:00]
Urząd Wojewódzki (Impart) [20:31:00] --- D ---> most Grunwaldzki [20:32:00]
most Grunwaldzki [20:32:00] --- D ---> PL. GRUNWALDZKI [20:34:00]
PL. GRUNWALDZKI [20:34:00] --- D ---> Ko

In [136]:
dijkstra(*test_cases[1])

Commute from Broniewskiego to Uniwersytet Ekonomiczny
Broniewskiego [10:16:00] --- 111 ---> Zakładowa [10:18:00]
Zakładowa [10:18:00] --- 111 ---> Słonimskiego [10:20:00]
Słonimskiego [10:20:00] --- 111 ---> Daszyńskiego [10:23:00]
Daszyńskiego [10:23:00] --- 23 ---> Nowowiejska [10:25:00]
Nowowiejska [10:25:00] --- 23 ---> Jedności Narodowej [10:26:00]
Jedności Narodowej [10:26:00] --- 23 ---> Na Szańcach [10:27:00]
Na Szańcach [10:27:00] --- 23 ---> pl. Bema [10:29:00]
pl. Bema [10:29:00] --- 23 ---> Hala Targowa [10:31:00]
Hala Targowa [10:31:00] --- 23 ---> pl. Nowy Targ [10:32:00]
pl. Nowy Targ [10:32:00] --- 23 ---> GALERIA DOMINIKAŃSKA [10:34:00]
GALERIA DOMINIKAŃSKA [10:35:00] --- 8 ---> Wzgórze Partyzantów [10:37:00]
Wzgórze Partyzantów [10:37:00] --- 8 ---> DWORZEC GŁÓWNY [10:39:00]
DWORZEC GŁÓWNY [10:39:00] --- 145 ---> DWORZEC AUTOBUSOWY [10:41:00]
DWORZEC AUTOBUSOWY [10:44:00] --- 10 ---> Sanocka [10:46:00]
Sanocka [10:46:00] --- 10 ---> Uniwersytet Ekonomiczny [10:48:00]


In [138]:
dijkstra(*test_cases[2])

Commute from POŚWIĘTNE to Młodych Techników at 15:30:00
POŚWIĘTNE [15:30:00] --- 7 ---> Wołowska [15:31:00]
Wołowska [15:31:00] --- 7 ---> Kępińska [15:32:00]
Kępińska [15:32:00] --- 7 ---> Kamieńskiego [15:33:00]
Kamieńskiego [15:33:00] --- 7 ---> Broniewskiego [15:35:00]
Broniewskiego [15:35:00] --- 7 ---> Trzebnicka [15:38:00]
Trzebnicka [15:38:00] --- 7 ---> DWORZEC NADODRZE [15:40:00]
DWORZEC NADODRZE [15:40:00] --- 7 ---> Paulińska [15:42:00]
Paulińska [15:42:00] --- 7 ---> Dubois [15:45:00]
Dubois [15:46:00] --- C  ---> Dmowskiego [15:49:00]
Dmowskiego [15:49:00] --- 144 ---> PL. JANA PAWŁA II [15:52:00]
PL. JANA PAWŁA II [15:52:00] --- 3 ---> Młodych Techników [15:54:00]
Total trip time is 00:24:00
Commuting takes 00:24:00
Algorithm took 11.68s to execute


Zmien czas by dawal czas przejazdu bez czekania na poczatku

In [144]:
dijkstra(*test_cases[3])

Commute from Śliczna to Marchewkowa at 23:50:00
Śliczna [23:53:00] --- 255 ---> Uniwersytet Ekonomiczny [23:55:00]
Uniwersytet Ekonomiczny [23:55:00] --- 255 ---> PETRUSEWICZA [23:56:00]
PETRUSEWICZA [24:00:00] --- 243 ---> DWORZEC AUTOBUSOWY [24:02:00]
DWORZEC AUTOBUSOWY [24:03:00] --- 241 ---> EPI [24:05:00]
EPI [24:05:00] --- 248 ---> Zaolziańska [24:06:00]
Zaolziańska [24:02:00] --- 32 ---> Wielka [24:03:00]
Wielka [24:03:00] --- 32 ---> Rondo [24:04:00]
Rondo [24:00:00] --- 3 ---> Sztabowa [24:01:00]
Sztabowa [24:01:00] --- 3 ---> Hallera [24:02:00]
Hallera [24:00:00] --- 7 ---> Jastrzębia [24:01:00]
Jastrzębia [24:00:00] --- 32 ---> Orla [24:01:00]
Orla [24:02:00] --- 7 ---> KRZYKI [24:05:00]
KRZYKI [24:05:00] --- 7 ---> Radio i Telewizja [24:06:00]
Radio i Telewizja [24:06:00] --- 7 ---> Przyjaźni [24:08:00]
Przyjaźni [24:08:00] --- 7 ---> Braterska [24:09:00]
Braterska [24:09:00] --- 7 ---> Sąsiedzka [24:10:00]
Sąsiedzka [24:10:00] --- 7 ---> KLECINA [24:11:00]
KLECINA [24:48:0

In [605]:
graph, goal_index, came_from, solution_cost, elapsed_time = run_solution(*test_cases[3])

In [606]:
came_from

{-1: None,
 12394: -1,
 254766: -1,
 256893: -1,
 226197: -1,
 256987: 254766,
 212092: 254766,
 254767: 254766,
 250184: 254767,
 252636: 254767,
 257603: 254767,
 85400: 226197,
 257605: 226197,
 226198: 226197,
 212120: 226198,
 226199: 226198,
 182642: 226199,
 180665: 226199,
 254396: 226199,
 254041: 226199,
 257428: 226199,
 226200: 226199,
 215921: 226199,
 261345: 226199,
 261427: 226199,
 252210: 250184,
 250185: 250184,
 65925: 250184,
 135980: 250184,
 251369: 250184,
 226201: 226200,
 256894: 256893,
 239034: 256893,
 215937: 256893,
 180648: 226201,
 180662: 226201,
 226202: 226201,
 16943: 250185,
 250186: 250185,
 254770: 250185,
 26806: 250185,
 125941: 250185,
 255264: 250185,
 256895: 256894,
 255064: 256894,
 226203: 226202,
 255642: 226202,
 250326: 251369,
 176607: 251369,
 254036: 251369,
 254397: 254396,
 256896: 256895,
 255208: 256895,
 226204: 226203,
 244153: 250186,
 250187: 250186,
 24630: 250186,
 254037: 254036,
 212121: 254397,
 180666: 254397,
 1210: 2

In [610]:
graph.conn_at_index(226197)

line                                  143
departure_time                   24:24:00
arrival_time                     24:27:00
start_stop                        Śliczna
end_stop          Uniwersytet Ekonomiczny
departure_sec                        1440
arrival_sec                          1620
Name: 226197, dtype: object

In [408]:
Graph(pd.read_csv(DATA_DIR / 'connection_graph_latest.csv', 
                               usecols=['line', 'departure_time', 'arrival_time', 'start_stop', 'end_stop'], 
                               dtype=str)).get_nodes()

array(['Zajezdnia Obornicka', 'Paprotna', 'Obornicka (Wołowska)',
       'Bezpieczna', 'Bałtycka', 'Broniewskiego', 'Pola', 'Syrokomli',
       'Kasprowicza', 'pl. Daniłowskiego', 'Przybyszewskiego',
       'Czajkowskiego', 'KOSZAROWA (Uniwersytet)', 'KOSZAROWA (Szpital)',
       'Berenta', 'KROMERA', 'Mosty Warszawskie', 'Wyszyńskiego',
       'Ogród Botaniczny', 'Katedra',
       'Urząd Wojewódzki (Muzeum Narodowe)', 'Poczta Główna',
       'skwer Krasińskiego', 'Wzgórze Partyzantów', 'Renoma',
       'Arkady (Capitol)', 'Pl. Hirszfelda', 'Krucza',
       'Krucza (Mielecka)', 'Inżynierska', 'Aleja Pracy', 'FAT',
       'GRABISZYŃSKA (Cmentarz)', 'Solskiego', 'Wiejska', 'Kadłubka',
       'Stanki', 'Bukowskiego', 'RACŁAWICKA', 'Rymarska', 'Wawrzyniaka',
       'Chłodna', 'Sowia', 'KRZYKI', 'Dworzec Główny (Dworcowa)',
       'Krasińskiego', 'Damrota', 'Koszarowa', 'Sołtysowicka',
       'Poprzeczna', 'Redycka', 'Bagatela', 'SOŁTYSOWICE', 'POŚWIĘTNE',
       'Wołowska', 'Kępińska', 'Ka